# TestQuery Notebook

This is a test Jupyter notebook to accompany the post:

<ul style="list-style-type: none;">
<li>[<b>Jupyter Notebooks with Elixir and RDF:</b><br/>Using IElixir in Jupyter Lab to explore SPARQL.ex](https://medium.com/@tonyhammond/jupyter-notebooks-with-elixir-and-rdf-598689c2dad3)
</ul>

And something, or other.

We shall be using [IElixir](https://github.com/pprzetacznik/IElixir) which provides the Jupyter kernel for Elixir. We'll also be using the Elixir package [SPARQL.ex](https://hex.pm/packages/sparql) which provides for SPARQL support. 


Here's the overview:

1. Setting up the environment
2. Simple queries
3. Install our TestQuery module
4. Testing it out
5. Etc.


## 1. Setting up the environment

&#x274F; ** Create `sparql_env`.**

IElixir uses the concept of virtual environments for managing packages. It uses [`Boyle`](https://github.com/pprzetacznik/IElixir#package-management-with-boyle) as its package manager.

Let's first create a `sparql_env` environment for our SPARQL dependencies using `Boyle.mk/1`. And note that we have previously created an `rdf_env` environment for exploring [RDF.ex](https://hex.pm/packages/rdf).

In [ ]:
Boyle.mk("sparql_env")

All dependencies up to date


{:ok, ["rdf_env", "sparql_env"]}

&#x274F; **Activate the environment.**

Next step is to activate the environment which will take care of compiling.

In [ ]:
Boyle.activate("sparql_env")

===> Compiling parse_trans
===> Compiling mimerl
===> Compiling metrics
===> Compiling unicode_util_compat
===> Compiling idna
===> Compiling certifi
===> Compiling hackney


:ok

And we can double check that this is the current environment.

In [ ]:
Boyle.active_env_name

"sparql_env"

&#x274F; **Install `sparql_client` dependencies.**

Next we install out dependencies.

In [ ]:
Boyle.install( {:sparql_client, "~> 0.2.1"})

Resolving Hex dependencies...
Dependency resolution completed:
Unchanged:
  content_type 0.1.0
  decimal 1.5.0
  elixir_uuid 1.2.0
  jason 1.1.2
  json_ld 0.3.0
  mime 1.3.0
  nimble_csv 0.4.0
  rdf 0.5.1
  sparql 0.2.5
  sparql_client 0.2.1
  sweet_xml 0.6.5
  tesla 1.1.0
All dependencies up to date


:ok

&#x274F; **And check our packages.**

And lastly let's just sanity check that we do indeed have the `SPARQL` and `SPARQL.Client` modules installed.

In [ ]:
exports SPARQL

execute_query/2                   execute_query/3                   query/1                           
query/2                           result_format/1                   result_format_by_extension/1      
result_format_by_media_type/1     result_formats/0                  


In [ ]:
exports SPARQL.Client

__adapter__/0               __middleware__/0            default_accept_header/1     
delete/1                    delete/2                    delete/3                    
delete!/1                   delete!/2                   delete!/3                   
get/1                       get/2                       get/3                       
get!/1                      get!/2                      get!/3                      
head/1                      head/2                      head/3                      
head!/1                     head!/2                     head!/3                     
options/1                   options/2                   options/3                   
options!/1                  options!/2                  options!/3                  
patch/2                     patch/3                     patch/4                     
patch!/2                    patch!/3                    patch!/4                    
post/2                      post/3                      post/4   

Looks good.

## 2. Simple queries

&#x274F; **Let's try out a simple query.**

Let's choose a SPARQL endpoint. DBpedia will be good.

In [ ]:
service = "http://dbpedia.org/sparql"

"http://dbpedia.org/sparql"

And we'll create a simple SPARQL query.

In [ ]:
query = "select * where {?s ?p ?o} limit 5"

"select * where {?s ?p ?o} limit 5"

&#x274F; **First query try gives error.**

In [ ]:
SPARQL.Client.query(query, service)

{:error, :econnrefused}

&#x274F; **Let's check out the documentation.**

In [ ]:
h SPARQL.Client.query


                   def query(query, endpoint, options \\ %{})                   

The query operation is used to send a SPARQL query to a service endpoint and
receive the results of the query.

The query can either be given as string or as an already parsed SPARQL.Query.

    with %SPARQL.Query{} = query <- SPARQL.Query.new("SELECT * WHERE { ?s ?p ?o }") do
      SPARQL.Client.query(query, "http://dbpedia.org/sparql")
    end

The type of the result returned depends on the query form:

  • SELECT queries will return a SPARQL.Query.ResultSet struct with a list
    of SPARQL.Query.Result structs in the results field.
  • ASK queries will return a SPARQL.Query.ResultSet struct with the
    boolean result in the results field
  • CONSTRUCT and DESCRIBE queries will return an RDF data structure

## Specifying the request method

The SPARQL 1.1 protocol spec defines three methods
(https://www.w3.org/TR/sparql11-protocol/#query-operation) to perform a SPARQL
query operation via HTTP, which c

&#x274F; **Second try succeeds – after setting :request_method (and :protocol_version).**

In [ ]:
SPARQL.Client.query(query, service, request_method: :get, protocol_version: "1.1")

{:ok, %SPARQL.Query.Result{results: [%{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid>}, %{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid-nullable>}, %{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid-nonblank>}, %{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid-nonblank-nullable>}, %{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax

## 3. Installing our TestQuery module

&#x274F; **Install TestQuery**

Is our `TestQuery` module loaded?

In [13]:
exports TestQuery

UndefinedFunctionError: 1

That'll be a no, then. So, let's explicitly import the module.

In [ ]:
import_file("/Users/tony/Projects/local/elixir/examples/test_query/lib/test_query.ex")

{:module, TestQuery, <<70, 79, 82, 49, 0, 0, 7, 76, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 191, 0, 0, 0, 18, 16, 69, 108, 105, 120, 105, 114, 46, 84, 101, 115, 116, 81, 117, 101, 114, 121, 8, 95, 95, 105, 110, 102, ...>>, {:query, 2}}

Rats!

Turns out that our call `:code.priv_dir(:test_query)}` is failing.

Let's just hardwire that for now and try again.

In [ ]:
import_file("/Users/tony/Projects/local/elixir/examples/test_query/lib/test_query.ex")

{:module, TestQuery, <<70, 79, 82, 49, 0, 0, 7, 76, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 191, 0, 0, 0, 18, 16, 69, 108, 105, 120, 105, 114, 46, 84, 101, 115, 116, 81, 117, 101, 114, 121, 8, 95, 95, 105, 110, 102, ...>>, {:query, 2}}

And let's sanity check.

In [ ]:
exports TestQuery

data/0      query/0     query/1     query/2     


Success!

&#x274F; **Install TestQuery.Client.** 

Now let's import the `TestQuery.Client` module.

In [17]:
import_file("/Users/tony/Projects/local/elixir/examples/test_query/lib/test_query/client.ex")

Protocol.UndefinedError: 1

Same problem. Same fix.

In [ ]:
import_file("/Users/tony/Projects/local/elixir/examples/test_query/lib/test_query/client.ex")

{:module, TestQuery.Client, <<70, 79, 82, 49, 0, 0, 25, 168, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 2, 195, 0, 0, 0, 69, 23, 69, 108, 105, 120, 105, 114, 46, 84, 101, 115, 116, 81, 117, 101, 114, 121, 46, 67, 108, 105, 101, 110, ...>>, {:_read_tuple, 1}}

And again sanity check.

In [ ]:
exports TestQuery.Client

get_query/0       get_service/0     hello/0           read_table/1      rquery/0          
rquery/1          rquery/2          rquery_all/0      


## 4. Testing it out

In [ ]:
o = SPARQL.Query.Result.get(result, :o) |> List.first

~L"Hello World"en

In [32]:
query

File.Error: 1

In [32]:
i query

File.Error: 1

In [ ]:
o

[~L"Hello World"en]

In [ ]:
o.value

"Hello World"

In [ ]:
rq = File.read!("/Users/tony/Projects/local/elixir/examples/test_super/priv/queries/dbpedia_query.rq")

"prefix dbo: <http://dbpedia.org/ontology/>\nprefix foaf: <http://xmlns.com/foaf/0.1/>\nprefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n\nselect *\nwhere {\n  bind (12345 as ?id)\n  ?s dbo:wikiPageID ?id .\n  optional { ?s foaf:isPrimaryTopicOf ?topic }\n  optional { ?s rdfs:label ?label }\n  filter (langMatches(lang(?label), \"en\"))\n} limit 1\n"

In [ ]:
IO.puts rq

prefix dbo: <http://dbpedia.org/ontology/>
prefix foaf: <http://xmlns.com/foaf/0.1/>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>

select *
where {
  bind (12345 as ?id)
  ?s dbo:wikiPageID ?id .
  optional { ?s foaf:isPrimaryTopicOf ?topic }
  optional { ?s rdfs:label ?label }
  filter (langMatches(lang(?label), "en"))
} limit 1



:ok

## 5. Etc.

In [ ]:
exports SPARQL.Query.Result

__struct__/0          __struct__/1          add_identity/1        append/2              
get/2                 new/1                 new/2                 remove_identity/1     


In [ ]:
 objects = (result |> SPARQL.Query.Result.get(:o) )

[~I<https://twitter.com/bgmarx>, ~I<https://twitter.com/josevalim>, ~I<https://twitter.com/redrapids>, %RDF.Literal{value: ~D[2018-03-14], datatype: ~I<http://www.w3.org/2001/XMLSchema#date>}, ~L"Paper", ~I<https://pragprog.com/>, ~L"Adopting Elixir"en, ~I<http://purl.org/ontology/bibo/Book>]